In [ ]:
from estival.model import BayesianCompartmentalModel
from estival import targets as est

import tbh.runner_tools as rt
from tbh.model import get_tb_model
from tbh.plotting import visualise_mle_params, plot_single_fit

In [ ]:
params, priors, tv_params = rt.get_parameters_and_priors()
model = get_tb_model(rt.DEFAULT_MODEL_CONFIG, tv_params)
bcm = BayesianCompartmentalModel(model, params, priors, rt.targets)

In [ ]:
import nevergrad as ng
from estival.wrappers.nevergrad import optimize_model

In [ ]:
opt_class = ng.optimizers.NGOpt
orunner = optimize_model(bcm, opt_class=opt_class)
rec = orunner.minimize(10000)

In [ ]:
mle_params = rec.value[1]
mle_params
res = bcm.run(mle_params)

In [ ]:
mle_params

In [ ]:
modified_params = mle_params | {
    "breakdown_rate": 0.5,
    "raw_transmission_rate": 1.0,
    "prev_se_cleared": 1.,
    "prev_se_contained": 1.,
    "prev_se_incipient": 1.,

    }
modified_res = bcm.run(modified_params)

In [ ]:
plot_single_fit(bcm, mle_params)

In [ ]:
plot_single_fit(bcm, modified_params)

In [ ]:
year = 2025
deno = res.derived_outputs['tb_prevalence'].loc[year]
for state in ["subclin_lowinf", "clin_lowinf", "subclin_inf", "clin_inf"]:
    prev = res.derived_outputs[f"prev_{state}"].loc[year]
    print(f"% {state}: {round(100 * prev / deno)}")

In [ ]:
year = 2025
deno = res.derived_outputs['tbi_prevalence'].loc[year]
for state in ["incipient", "contained", "cleared"]:
    prev = res.derived_outputs[f"prev_{state}"].loc[year]
    print(f"% {state}: {round(100 * prev / deno)}")

In [ ]:
visualise_mle_params(bcm.priors, mle_params)